<a href="https://colab.research.google.com/github/nitin-vig/ERAv4S5/blob/main/MNIST_Training_in_less_than_20k_Params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Training in less than 20k Params

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 4, 3)          # Output: [4, 26, 26]
        self.bn1 = nn.BatchNorm2d(4)

        self.conv2 = nn.Conv2d(4, 8, 3)          # Output: [8, 24, 24]
        self.bn2 = nn.BatchNorm2d(8)

        self.pool1 = nn.MaxPool2d(2, 2)          # Output: [8, 12, 12]
        self.dropout1 = nn.Dropout2d(0.1)

        self.conv3 = nn.Conv2d(8, 16, 3)         # Output: [16, 10, 10]
        self.bn3 = nn.BatchNorm2d(16)

        self.conv4 = nn.Conv2d(16, 32, 3)        # Output: [32, 8, 8]
        self.bn4 = nn.BatchNorm2d(32)

        self.pool2 = nn.MaxPool2d(2, 2)          # Output: [32, 4, 4]
        self.dropout2 = nn.Dropout2d(0.05)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 4 * 4, 25)
        self.bn_fc1 = nn.BatchNorm1d(20)
        self.dropout_fc = nn.Dropout(0.3) ## Learning: Experimenteed with 0.2 vs 0.3 ########

        self.out = nn.Linear(25, 10)

    def forward(self, x):
        # Convolutional Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        # x = self.dropout1(x) ##Learning: no dropout as number of channels is low

        # Convolutional Block 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        # x = self.dropout2(x)

        # Flatten
        x = x.view(-1, 32 * 4 * 4)

        # Fully Connected Block
        x = F.relu((self.fc1(x)))
        x = self.dropout_fc(x)

        # Output Layer
        x = self.out(x)
        return F.log_softmax(x, dim=1)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              40
       BatchNorm2d-2            [-1, 4, 26, 26]               8
            Conv2d-3            [-1, 8, 24, 24]             296
       BatchNorm2d-4            [-1, 8, 24, 24]              16
         MaxPool2d-5            [-1, 8, 12, 12]               0
            Conv2d-6           [-1, 16, 10, 10]           1,168
       BatchNorm2d-7           [-1, 16, 10, 10]              32
            Conv2d-8             [-1, 32, 8, 8]           4,640
       BatchNorm2d-9             [-1, 32, 8, 8]              64
        MaxPool2d-10             [-1, 32, 4, 4]               0
           Linear-11                   [-1, 25]          12,825
          Dropout-12                   [-1, 25]               0
           Linear-13                   [-1, 10]             260
Total params: 19,349
Trainable params: 

In [4]:

torch.manual_seed(1)
batch_size = 128 ## Learning: Reduce batch size for better learning but also increased the training time.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(10),
                        transforms.RandomAffine(0, translate=(0.1, 0.1)), ##Learning: Added transformation for better generalization
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.09MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.30MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.30MB/s]


In [5]:
# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5) ## Learning: adjust LR to a smaller value after every 5 epochs for fine tuning


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    # Update the learning rate
    scheduler.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:


for epoch in range(1, 21):
    print(f"\n Epoch: {epoch} : ")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

# plt.plot(test_loss,label='test_loss')
# plt.legend()
# plt.show


 Epoch: 1 : 


loss=0.2783926725387573 batch_id=468: 100%|██████████| 469/469 [00:56<00:00,  8.36it/s]


Test set: Average loss: 0.0851, Accuracy: 9724/10000 (97.24%)


 Epoch: 2 : 


loss=0.16270093619823456 batch_id=468: 100%|██████████| 469/469 [00:51<00:00,  9.19it/s]


Test set: Average loss: 0.0417, Accuracy: 9864/10000 (98.64%)


 Epoch: 3 : 


...

Test set: Average loss: 0.0190, Accuracy: 9941/10000 (99.41%)


 Epoch: 19 : 


loss=0.07043924182653427 batch_id=468: 100%|██████████| 469/469 [00:51<00:00,  9.02it/s]


Test set: Average loss: 0.0205, Accuracy: 9937/10000 (99.37%)


 Epoch: 20 : 


loss=0.06360998004674911 batch_id=468: 100%|██████████| 469/469 [00:51<00:00,  9.14it/s]


Test set: Average loss: 0.0187, Accuracy: 9940/10000 (99.40%)

